## Create & Access SQLite database using Python

In [1]:
import sqlite3

Creating a database and establishing the connection

In [2]:
conn = sqlite3.connect("UFC_competitors.db")
# Pass ":memory:" to open a connection to a database that is in RAM instead of on disk.

In [3]:
cursor = conn.cursor()

Creating a table

In [4]:
cursor.execute("DROP TABLE IF EXISTS competitors") 

table_competitors = """
CREATE TABLE competitors 
(id INTEGER PRIMARY KEY AUTOINCREMENT,
f_name VARCHAR(15) NOT NULL,
l_name VARCHAR(15) NOT NULL,
weight_class INTEGER);
"""

cursor.execute(table_competitors)

print("Created table competitors")

Created table competitors


Inserting data into a table using placeholders instead of string formatting to bind Python values to SQL statements and avoid SQL injection attacks

In [5]:
data = [
(1, "Jan", "Blachowicz", 205),
(2, "Aleksander", "Volkanovski", 145),
(3, "Amanda", "Nunes", 135),
(4, "Stephen", "Thompson", 170),
(5, "Dustin", "Poirier", 155),
]

cursor.executemany("""
INSERT INTO competitors VALUES (?, ?, ?, ?)""", data)

conn.commit()

Query data in the table

In [6]:
query = "SELECT * FROM competitors"
cursor.execute(query)
three_rows = cursor.fetchmany(3)
for x in three_rows:
    print(x) 

print("The rest of the cursor content:")
cursor.fetchall()

(1, 'Jan', 'Blachowicz', 205)
(2, 'Aleksander', 'Volkanovski', 145)
(3, 'Amanda', 'Nunes', 135)
The rest of the cursor content:


[(4, 'Stephen', 'Thompson', 170), (5, 'Dustin', 'Poirier', 155)]

Update table

In [7]:
update = "UPDATE competitors SET weight_class = 145 where id = 3"
cursor.execute(update)
conn.commit()

In [8]:
cursor.execute("SELECT * FROM competitors where id = 3")
cursor.fetchone()

(3, 'Amanda', 'Nunes', 145)

### Retrieve data into pandas DataFrame

In [9]:
import pandas as pd

read_sql_query method

In [10]:
df = pd.read_sql_query("SELECT * FROM competitors WHERE weight_class > 145", conn, index_col = "id")
df

,f_name,l_name,weight_class
id,,,
1,Jan,Blachowicz,205
4,Stephen,Thompson,170
5,Dustin,Poirier,155


##### read_sql_table method

When retrieving a table, specify 'con' parameter as URI or SQLAlchemy connectable.

SQLite DBAPI connection mode not supported.

In [11]:
import sqlalchemy
engine = sqlalchemy.create_engine("sqlite:///UFC_competitors.db")
connection = engine.connect()

In [12]:
df2 = pd.read_sql_table("competitors", connection, index_col="id")
df2

,f_name,l_name,weight_class
id,,,
1,Jan,Blachowicz,205
2,Aleksander,Volkanovski,145
3,Amanda,Nunes,145
4,Stephen,Thompson,170
5,Dustin,Poirier,155


In [13]:
df2 = pd.read_sql_table("competitors", "sqlite:///UFC_competitors.db", index_col="id")
df2

,f_name,l_name,weight_class
id,,,
1,Jan,Blachowicz,205
2,Aleksander,Volkanovski,145
3,Amanda,Nunes,145
4,Stephen,Thompson,170
5,Dustin,Poirier,155


##### read_sql method
can be used both for queries and for table retrieval

In [14]:
# read_sql for querry:
df = pd.read_sql("SELECT l_name, weight_class FROM competitors", conn)
df

,l_name,weight_class
0,Blachowicz,205
1,Volkanovski,145
2,Nunes,145
3,Thompson,170
4,Poirier,155


In [15]:
# read_sql for table.'Con' parameter defined with the URI:
df2 = pd.read_sql("competitors", con="sqlite:///UFC_competitors.db", index_col="id")
df2

,f_name,l_name,weight_class
id,,,
1,Jan,Blachowicz,205
2,Aleksander,Volkanovski,145
3,Amanda,Nunes,145
4,Stephen,Thompson,170
5,Dustin,Poirier,155


In [16]:
# read_sql for table. 'Con' parameter defined with SQLAlchemy connection:
df2 = pd.read_sql("competitors", con=connection, index_col="id")
df2

,f_name,l_name,weight_class
id,,,
1,Jan,Blachowicz,205
2,Aleksander,Volkanovski,145
3,Amanda,Nunes,145
4,Stephen,Thompson,170
5,Dustin,Poirier,155


In [17]:
conn.close()
connection.close()

### SQL Magic

To communicate with SQL Databases from within a JupyterLab notebook, you can use the SQL "magic" provided by the ipython-sql extension. First install the extension and ibm_db_sa driver.

In [18]:
!pip install ipython-sql
!pip install ibm_db_sa

Then load the SQL extension:

In [19]:
%load_ext sql

Establishing the connection with the database:

In [20]:
%sql sqlite:///UFC_competitors.db

%%sql at the top of a cell allows for the entire cell to be treated as SQL.

Python variables can be used in SQL statements by adding a ":" prefix to python variable names.

In [21]:
light_heavyweight  = 205
%sql select * from competitors where weight_class = :light_heavyweight

 * sqlite:///UFC_competitors.db
Done.


id,f_name,l_name,weight_class
1,Jan,Blachowicz,205


Python assignment syntax can be used to assign the results of queries to python variables.

In [22]:
Last_names = %sql select l_name as Last_name from competitors;
Last_names

 * sqlite:///UFC_competitors.db
Done.


Last_name
Blachowicz
Volkanovski
Nunes
Thompson
Poirier


In [23]:
Last_names = Last_names.DataFrame()

Analyzing a CSV file using pandas and SQL magic:

In [24]:
con = sqlite3.connect("database_name.db")
cur = con.cursor()

In [25]:
df = pandas.read_csv('file_name.csv')
df.to_sql("table_name", con, if_exists='replace', index=False,method="multi")

NameError: name 'pandas' is not defined

Get tables' names

In [ ]:
%sql select name from sqlite_master where type='table';

In [ ]:
# Query to retrieve the number of columns in the SCHOOLS table
%sql select count(name) from PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');b

In [ ]:
# Query to retrieve all column names in the SCHOOLS table along with their datatypes and length
%sql SELECT name,type,length(type) FROM PRAGMA_TABLE_INFO('CHICAGO_PUBLIC_SCHOOLS_DATA');